In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [53]:
"""
read in our data file, get the unique set of characters that we see
"""

with open('wizard_of_oz.txt', 'r', encoding='utf-8')as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [54]:
"""
character based tokenization 
encoding and decoding (create a mapping from char to int)
tensor -> multidimensional array 
set to torch.long (64-bit integer) for later
"""

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [55]:
print(data[:100])

tensor([80, 44, 61, 58,  1, 40, 71, 68, 63, 58, 56, 73,  1, 31, 74, 73, 58, 67,
        55, 58, 71, 60,  1, 58, 26, 68, 68, 64,  1, 68, 59,  1, 28, 68, 71, 68,
        73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71, 57,
         1, 62, 67,  1, 39, 79,  0,  0, 44, 62, 73, 65, 58, 22,  1, 28, 68, 71,
        68, 73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71,
        57,  1, 62, 67,  1, 39, 79,  0,  0, 25])


In [56]:
"""
split into training/validation datasets
"""
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split='train'):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([107837,  44238,  89562,  70893])
inputs:
tensor([[54, 74, 72, 58,  1, 73, 61, 58],
        [56, 58,  1, 76, 61, 58, 71, 58],
        [54, 67, 57,  1, 56, 61, 62, 65],
        [28,  1, 44, 32, 29,  1, 47, 33]])
targets:
tensor([[74, 72, 58,  1, 73, 61, 58, 78],
        [58,  1, 76, 61, 58, 71, 58,  1],
        [67, 57,  1, 56, 61, 62, 65, 57],
        [ 1, 44, 32, 29,  1, 47, 33, 50]])


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        return logits